In [32]:
import pandas as pd

# Load the data
data = pd.read_csv("Training.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]

# Split the data into features and target
features = data.drop('prognosis', axis=1)
target = data['prognosis']

In [34]:
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler
scaler = MinMaxScaler()

# Fit and transform the features
features = scaler.fit_transform(features)


In [35]:
from sklearn.preprocessing import LabelEncoder

# Initialize a label encoder
le = LabelEncoder()

# Fit and transform the target
target = le.fit_transform(target)


In [36]:
import torch
from torch import nn

# Define the neural network
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(132, 16)  # input layer
        self.fc2 = nn.Linear(16, 16)   # hidden layer 1
        self.fc3 = nn.Linear(16, 16)   # hidden layer 2
        self.fc4 = nn.Linear(16, 41)   # output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.log_softmax(self.fc4(x), dim=1)
        return x

model = Net()


In [42]:
# Convert the data to PyTorch tensors
features_tensor = torch.tensor(features, dtype=torch.float32)
target_tensor = torch.tensor(target, dtype=torch.long)

# Split the data into training and validation sets
train_size = int(0.8 * len(features_tensor))
val_size = len(features_tensor) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(list(zip(features_tensor, target_tensor)), [train_size, val_size])

# Define the data loaders
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=True)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Train the model
num_epochs = 1000
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print loss for every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    if(loss.item() < 0.001):
        break


Epoch [1/1000], Loss: 3.7113
Epoch [2/1000], Loss: 3.7183
Epoch [3/1000], Loss: 3.7042
Epoch [4/1000], Loss: 3.6701
Epoch [5/1000], Loss: 3.7245
Epoch [6/1000], Loss: 3.6923
Epoch [7/1000], Loss: 3.7172
Epoch [8/1000], Loss: 3.7354
Epoch [9/1000], Loss: 3.6989
Epoch [10/1000], Loss: 3.6723
Epoch [11/1000], Loss: 3.6696
Epoch [12/1000], Loss: 3.7237
Epoch [13/1000], Loss: 3.7041
Epoch [14/1000], Loss: 3.6951
Epoch [15/1000], Loss: 3.7009
Epoch [16/1000], Loss: 3.6845
Epoch [17/1000], Loss: 3.6808
Epoch [18/1000], Loss: 3.7105
Epoch [19/1000], Loss: 3.6761
Epoch [20/1000], Loss: 3.6866
Epoch [21/1000], Loss: 3.6490
Epoch [22/1000], Loss: 3.7210
Epoch [23/1000], Loss: 3.6910
Epoch [24/1000], Loss: 3.6815
Epoch [25/1000], Loss: 3.6420
Epoch [26/1000], Loss: 3.6605
Epoch [27/1000], Loss: 3.6680
Epoch [28/1000], Loss: 3.6400
Epoch [29/1000], Loss: 3.6172
Epoch [30/1000], Loss: 3.6137
Epoch [31/1000], Loss: 3.6141
Epoch [32/1000], Loss: 3.5520
Epoch [33/1000], Loss: 3.5555
Epoch [34/1000], Lo

In [43]:
# Load the test data
test_data = pd.read_csv("Testing.csv")
test_data = test_data.loc[:, ~test_data.columns.str.contains('^Unnamed')]

test_features = test_data.drop('prognosis', axis=1)
test_target = test_data['prognosis']

# Normalize the test features
test_features = scaler.transform(test_features)

test_target = le.fit_transform(test_target)


# Convert the test data to PyTorch tensors
test_features_tensor = torch.tensor(test_features, dtype=torch.float32)
test_target_tensor = torch.tensor(test_target, dtype=torch.long)

# Define the data loader for the test data
test_loader = torch.utils.data.DataLoader(list(zip(test_features_tensor, test_target_tensor)), batch_size=64, shuffle=False)

# Evaluate the model on the test data
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print(f'Accuracy of the model on the test data: {100 * correct / total}%')


42
42
Accuracy of the model on the test data: 100.0%
